In [1]:
import sys
sys.path.append('./tools/experiment')

import experiment
import executor

In [2]:
stress_exec =  executor.StressExecutor(opt_interval=1)
workload_exec =  executor.WorkloadExecutor(opt_interval=1)

In [3]:
exp = experiment.Experiment(
    mode="standard",
    ).with_mem_stress(
        memrate_range = range(1, 16 + 1),
    ).with_cpu_stress(
        cpu_range = [1, 2, 3 ,4],
        cpuload_range = [25, 50, 75, 100],
    ).with_workload(workloads=["redis"])

with exp:
    exp.run(stress_exec=stress_exec, workload_exec=workload_exec, interval=1)
    
exp.to_dict()

[INFO] 20230927093951 experiment start
[INFO] stress_executor: prologue
[INFO] stress_executor: exec {'mem': {'memrate': 1, 'memrate-bytes': '1G'}, 'cpu': {'cpu': 1, 'cpu-load': 25}}
[INFO] workload_executor: prologue
[INFO] workload_executor: exec {'mode': 'standard', 'workload': 'redis'}
[INFO] workload_executor: epilogue
[INFO] stress_executor: epilogue
[INFO] stress_executor: prologue
[INFO] stress_executor: exec {'mem': {'memrate': 2, 'memrate-bytes': '1G'}, 'cpu': {'cpu': 1, 'cpu-load': 50}}
[INFO] workload_executor: prologue
[INFO] workload_executor: exec {'mode': 'standard', 'workload': 'redis'}
[INFO] workload_executor: epilogue
[INFO] stress_executor: epilogue
[INFO] stress_executor: prologue
[INFO] stress_executor: exec {'mem': {'memrate': 3, 'memrate-bytes': '1G'}, 'cpu': {'cpu': 1, 'cpu-load': 75}}
[INFO] workload_executor: prologue
[INFO] workload_executor: exec {'mode': 'standard', 'workload': 'redis'}
[INFO] workload_executor: epilogue
[INFO] stress_executor: epilogue
[

{'mode': 'standard',
 'start_time': '20230927093951',
 'end_time': '20230927094143',
 'stress_per_epoch': [{'mem': {'memrate': 1, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 1, 'cpu-load': 25}},
  {'mem': {'memrate': 2, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 1, 'cpu-load': 50}},
  {'mem': {'memrate': 3, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 1, 'cpu-load': 75}},
  {'mem': {'memrate': 4, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 1, 'cpu-load': 100}},
  {'mem': {'memrate': 5, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 2, 'cpu-load': 25}},
  {'mem': {'memrate': 6, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 2, 'cpu-load': 50}},
  {'mem': {'memrate': 7, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 2, 'cpu-load': 75}},
  {'mem': {'memrate': 8, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 2, 'cpu-load': 100}},
  {'mem': {'memrate': 9, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 3, 'cpu-load': 25}},
  {'mem': {'memrate': 10, 'memrate-bytes': '1G'},
   'cpu': {'cpu': 3, 'cpu-load': 50}},
  {'mem': {'